# 第七章：建立聊天應用程式
## Github Models API 快速入門

本筆記本改編自 [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)，當中包含了可以存取 [Azure OpenAI](notebook-azure-openai.ipynb) 服務的筆記本。


# 概覽  
「大型語言模型其實就係一個將文字對應到文字嘅函數。當你輸入一段文字，大型語言模型會嘗試預測下一段會出現咩文字」(1)。呢份「快速入門」筆記會為用戶介紹高層次嘅LLM概念、開始使用AML所需嘅核心套件、簡單介紹提示設計，仲有幾個唔同應用場景嘅短例子。


## 目錄

[概覽](../../../../07-building-chat-applications/python)  
[如何使用 OpenAI 服務](../../../../07-building-chat-applications/python)  
[1. 建立你的 OpenAI 服務](../../../../07-building-chat-applications/python)  
[2. 安裝](../../../../07-building-chat-applications/python)  
[3. 認證資料](../../../../07-building-chat-applications/python)  

[使用案例](../../../../07-building-chat-applications/python)  
[1. 文字摘要](../../../../07-building-chat-applications/python)  
[2. 文字分類](../../../../07-building-chat-applications/python)  
[3. 產生新產品名稱](../../../../07-building-chat-applications/python)  
[4. 微調分類器](../../../../07-building-chat-applications/python)  

[參考資料](../../../../07-building-chat-applications/python)


### 建立你的第一個提示  
這個簡短練習會為你介紹如何在 Github Models 提交提示，完成一個簡單的「摘要」任務。

**步驟**:  
1. 如果你未安裝過，請先在你的 python 環境安裝 `azure-ai-inference` 套件。  
2. 載入標準輔助函式庫，並設定 Github Models 的認證。  
3. 選擇一個適合你任務的模型  
4. 為模型建立一個簡單的提示  
5. 將你的請求提交到模型 API！


### 1. 安裝 `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. 搵啱嘅模型  
GPT-3.5-turbo 或 GPT-4 模型可以理解同產生自然語言。


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. 提示設計

「大型語言模型的神奇之處在於，透過在大量文本上訓練以減少預測錯誤，模型最終會學到對這些預測有用的概念。例如，佢哋會學識以下概念」(1):

* 點樣串字
* 文法點運作
* 點樣改寫句子
* 點樣答問題
* 點樣同人對話
* 點樣用唔同語言寫作
* 點樣寫程式
* 等等

#### 點樣控制大型語言模型
「喺所有輸入大型語言模型嘅方法之中，最有影響力嘅就係文字提示」(1)。

大型語言模型可以用幾種方式提示去產生輸出：

指令：直接話俾模型知你想要咩
補全：引導模型去補完你想要嘅開頭
示範：用以下方法俾模型睇你想要咩：
提示入面有幾個例子
喺微調訓練數據集入面有幾百甚至幾千個例子

#### 創建提示有三個基本指引：

**展示同講明**。無論用指令、例子，定係兩者結合，都要清楚表達你想要咩。如果你想模型將一個清單按字母順序排列，或者將一段文字按情感分類，就要俾佢睇到你想要咁做。

**提供高質素數據**。如果你想建立一個分類器，或者想模型跟住某個模式行，記住要有足夠例子。記得校對你嘅例子——模型通常夠聰明，可以識穿基本嘅串字錯誤，照樣俾你回應，但佢有時都會以為你係特登咁做，咁就會影響回應。

**檢查你嘅設定。** temperature 同 top_p 呢兩個設定會影響模型產生回應時有幾大隨機性。如果你問嘅問題只得一個正確答案，就應該將呢啲設定調低。如果你想要多啲唔同嘅回應，就可以調高啲。最多人犯嘅錯誤就係以為呢啲設定係「聰明度」或者「創意」控制。

來源：https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 摘要文字  
#### 挑戰  
透過在文字段落結尾加上「tl;dr:」來進行摘要。留意模型如何在沒有額外指示下完成多項任務。你可以嘗試用更具描述性的提示詞來調整模型行為，從而自訂你想要的摘要效果(3)。

最近的研究顯示，先在大量文本語料庫上進行預訓練，再針對特定任務微調，可以在許多 NLP 任務和基準上取得顯著進步。雖然這種方法的架構通常與任務無關，但仍然需要數千甚至數萬個任務專屬的微調數據集。相比之下，人類通常只需少量例子或簡單指示就能完成新的語言任務——這是現有 NLP 系統仍然難以做到的。我們發現，擴大語言模型規模能大幅提升無需任務專屬微調的少量示例表現，有時甚至能與過去最先進的微調方法媲美。



Tl;dr


# 多個使用情境的練習  
1. 摘要文本  
2. 分類文本  
3. 產生新產品名稱


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 分類文本  
#### 挑戰  
將項目分類到推理時提供的類別中。在以下例子中，我們會在提示中同時提供類別和需要分類的文本（*playground_reference）。

客戶查詢：你好，我手提電腦鍵盤有一粒鍵最近壞咗，我需要換一粒新嘅：

分類類別：


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 產生新產品名稱
#### 挑戰
根據範例字詞創作產品名稱。我哋會喺提示入面加入關於要為邊款產品起名嘅資訊，亦會提供一個類似嘅例子，展示我哋期望收到嘅命名模式。我哋亦將 temperature 數值調高，令結果更有創意同多變。

產品描述：家用奶昔機
種子字詞：快、健康、細小。
產品名稱：HomeShaker、Fit Shaker、QuickShake、Shake Maker

產品描述：一對可以適合任何腳型嘅鞋。
種子字詞：適應、合腳、全適配。


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# 參考資料  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 範例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [微調 GPT-3 以分類文本的最佳實踐](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# 需要更多協助

[OpenAI 商業化團隊](AzureOpenAITeam@microsoft.com)


# 貢獻者
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人工翻譯。本翻譯所引致的任何誤解或誤釋，我們概不負責。
